In [1]:
#importing libaries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq, fftshift

In [2]:
bearing_signals = pd.read_csv('bearing_signals.csv')
bearing_classes = pd.read_csv('bearing_classes.csv', delimiter=';')

In [3]:
bearing_signals = bearing_signals.drop(['bearing_1_id','experiment_id'], 1)
bearing_signals.rename(columns={'bearing_2_id': 'bearing_id'}, inplace=True)
bearing_signals.head()

,bearing_id,timestamp,a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,rpm,hz,w
0,1,0.000000,0.113269,0.149706,-0.110275,-0.186030,0.194450,0.454299,0.0,0.0,0.000006
1,1,0.000333,-0.367713,-0.228832,0.177821,0.285992,0.002226,-0.043930,0.0,0.0,0.000243
2,1,0.000667,0.113269,0.149706,-0.398371,-0.091625,0.002226,0.454299,0.0,0.0,0.000369
3,1,0.001000,-0.175320,-0.228832,-0.110275,0.285992,0.002226,0.255007,0.0,0.0,0.000520
4,1,0.001333,-0.079124,0.055072,-0.110275,0.191588,0.002226,0.255007,0.0,0.0,0.000175


In [4]:
df = (pd.merge(bearing_signals, bearing_classes, on='bearing_id'))
df.head(2)

,bearing_id,timestamp,a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,rpm,hz,w,status
0,1,0.000000,0.113269,0.149706,-0.110275,-0.186030,0.194450,0.454299,0.0,0.0,0.000006,0
1,1,0.000333,-0.367713,-0.228832,0.177821,0.285992,0.002226,-0.043930,0.0,0.0,0.000243,0


Calculating metrics per bearing

In [5]:
df_mean = df.groupby('bearing_id').mean()
df_mean.columns = ['timestamp','a1_x_mean','a1_y_mean','a1_z_mean','a2_x_mean','a2_y_mean','a2_z_mean','rpm_mean','hz_mean','w_mean','status']
df_mean = df_mean.drop(['timestamp'], 1)
df_mean = df_mean.drop(40)

In [6]:
df_max = df.groupby('bearing_id').max()
df_max = df_max.drop(40)
df_max = df_max.drop(['timestamp','rpm','hz','status'], 1)
df_max.columns = ['a1_x_max','a1_y_max','a1_z_max','a2_x_max','a2_y_max','a2_z_max','w_max']

In [7]:
df_min = df.groupby('bearing_id').min()
df_min = df_min.drop(40)
df_min = df_min.drop(['timestamp','rpm','hz','w','status'], 1)
df_min.columns = ['a1_x_min','a1_y_min','a1_z_min','a2_x_min','a2_y_min','a2_z_min']

In [8]:
df_min_range = df.groupby('bearing_id').min()
df_min_range = df_min_range.drop(40)
df_min_range = df_min_range.drop(['timestamp','rpm','hz','status'], 1)

In [9]:
df_max_range = df.groupby('bearing_id').max()
df_max_range = df_max_range.drop(40)
df_max_range = df_max_range.drop(['timestamp','rpm','hz','status'], 1)

In [10]:
df_range = df_max_range.subtract(df_min_range, fill_value=0)
df_range.columns = ['a1_x_range','a1_y_range','a1_z_range','a2_x_range','a2_y_range','a2_z_range','w_range']
df_range

,a1_x_range,a1_y_range,a1_z_range,a2_x_range,a2_y_range,a2_z_range,w_range
bearing_id,,,,,,,
1,16.738155,9.936622,17.861935,18.692052,17.876810,22.021738,1.679265
2,15.006622,9.558084,16.421456,17.748009,16.723468,20.028821,1.907449
3,15.583800,9.558084,16.421456,21.712990,18.645705,19.132008,2.132677
4,12.505518,9.558084,15.173042,17.653605,15.666237,18.932716,1.877710
5,14.718033,10.882967,17.189712,20.580138,18.549593,17.039445,1.578119
...,...,...,...,...,...,...,...
108,15.583800,13.627367,20.166701,21.996202,15.474013,23.516426,1.887597
109,14.429444,10.788333,19.014318,14.349454,12.686769,21.124925,1.754798
110,19.624044,10.504429,19.494477,16.331944,13.071216,21.423863,1.707395


In [11]:
df_master = (pd.merge(df_mean, df_range, on='bearing_id'))
df_master = (pd.merge(df_master, df_min, on='bearing_id'))
df_master = (pd.merge(df_master, df_max, on='bearing_id'))
df_master

,a1_x_mean,a1_y_mean,a1_z_mean,a2_x_mean,a2_y_mean,a2_z_mean,rpm_mean,hz_mean,w_mean,status,...,a2_x_min,a2_y_min,a2_z_min,a1_x_max,a1_y_max,a1_z_max,a2_x_max,a2_y_max,a2_z_max,w_max
bearing_id,,,,,,,,,,,,,,,,,,,,,
1,-0.176971,-0.001740,-0.035361,0.114590,0.023329,0.250710,1082.423656,18.040394,0.419961,0,...,-10.004077,-8.647843,-10.207810,7.327991,5.638507,9.204820,8.687975,9.228967,11.813929,1.679266
2,-0.174032,-0.007636,-0.053353,-0.178986,0.024936,0.194061,1074.981634,17.916361,0.501160,0,...,-9.437651,-7.975060,-11.104623,7.424187,4.881431,8.340533,8.310358,8.748408,8.924198,1.907451
3,-0.172328,-0.002343,-0.039898,-0.121842,0.036149,0.183881,1149.927038,19.165451,0.513051,0,...,-12.080972,-8.455619,-11.204268,7.231794,5.449238,8.052437,9.632018,10.190086,7.927740,2.132678
4,-0.149644,-0.000549,-0.037679,-0.119574,0.030838,0.200699,1127.538630,18.792311,0.539426,0,...,-10.004077,-7.782836,-11.104623,5.981243,4.881431,8.148469,7.649527,7.883401,7.828094,1.877711
5,-0.146438,0.015268,-0.026678,-0.080799,0.034042,0.206547,1037.216452,17.286941,0.353715,0,...,-11.325737,-8.840067,-7.816309,6.750813,6.206314,8.340533,9.254401,9.709526,9.223136,1.578121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,-0.145112,0.026677,0.120057,-0.225940,0.069918,0.219440,952.405287,15.873421,0.387498,1,...,-11.136929,-6.917829,-12.101081,7.231794,6.679486,9.204820,10.859274,8.556184,11.415345,1.887599
109,-0.124200,0.077907,0.219025,-0.229740,0.099719,0.184505,1000.562215,16.676037,0.262752,1,...,-7.266352,-6.052822,-10.805685,7.520383,4.786797,9.012756,7.083102,6.633946,10.319241,1.754799
110,-0.132581,0.080428,0.213349,-0.209128,0.117938,0.079041,1073.939895,17.898998,0.302760,1,...,-8.682417,-6.052822,-9.709580,10.598665,4.313624,10.069107,7.649527,7.018394,11.714283,1.707396


In [12]:
df_master.columns.to_list()

['a1_x_mean',
 'a1_y_mean',
 'a1_z_mean',
 'a2_x_mean',
 'a2_y_mean',
 'a2_z_mean',
 'rpm_mean',
 'hz_mean',
 'w_mean',
 'status',
 'a1_x_range',
 'a1_y_range',
 'a1_z_range',
 'a2_x_range',
 'a2_y_range',
 'a2_z_range',
 'w_range',
 'a1_x_min',
 'a1_y_min',
 'a1_z_min',
 'a2_x_min',
 'a2_y_min',
 'a2_z_min',
 'a1_x_max',
 'a1_y_max',
 'a1_z_max',
 'a2_x_max',
 'a2_y_max',
 'a2_z_max',
 'w_max']

Engeneering frequencies

In [13]:
df_fft = df
acceleration_cols = ['a1_x','a1_y','a1_z','a2_x','a2_y','a2_z']
acceleration_fft = fft(df[acceleration_cols].values)
df_fft['a1_x_fft'] = abs(acceleration_fft[:,0])
df_fft['a1_y_fft'] = abs(acceleration_fft[:,1])
df_fft['a1_z_fft'] = abs(acceleration_fft[:,2])
df_fft['a2_x_fft'] = abs(acceleration_fft[:,3])
df_fft['a2_y_fft'] = abs(acceleration_fft[:,4])
df_fft['a2_z_fft'] = abs(acceleration_fft[:,5])
df_fft.head()

,bearing_id,timestamp,a1_x,a1_y,a1_z,a2_x,a2_y,a2_z,rpm,hz,w,status,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
0,1,0.000000,0.113269,0.149706,-0.110275,-0.186030,0.194450,0.454299,0.0,0.0,0.000006,0,0.615419,0.768876,0.416851,0.220532,0.416851,0.768876
1,1,0.000333,-0.367713,-0.228832,0.177821,0.285992,0.002226,-0.043930,0.0,0.0,0.000243,0,0.174436,0.880146,0.314195,0.200896,0.314195,0.880146
2,1,0.000667,0.113269,0.149706,-0.398371,-0.091625,0.002226,0.454299,0.0,0.0,0.000369,0,0.229504,0.932711,0.116978,0.795256,0.116978,0.932711
3,1,0.001000,-0.175320,-0.228832,-0.110275,0.285992,0.002226,0.255007,0.0,0.0,0.000520,0,0.028798,0.649700,0.355533,0.595537,0.355533,0.649700
4,1,0.001333,-0.079124,0.055072,-0.110275,0.191588,0.002226,0.255007,0.0,0.0,0.000175,0,0.314494,0.277512,0.076581,0.688839,0.076581,0.277512


Merging calculated metrics with fequencies per bearing

In [14]:
df_fft = df_fft.drop(['timestamp', 'a1_x', 'a1_y', 'a1_z', 'a2_x', 'a2_y', 'a2_z', 'rpm', 'hz', 'w', 'status'], 1)
df_fft = df_fft.drop(40)

In [15]:
df_fft_mean = df_fft.groupby('bearing_id').mean()
df_fft_mean.columns = ['a1_x_fft_mean', 'a1_y_fft_mean', 'a1_z_fft_mean', 'a2_x_fft_mean', 'a2_y_fft_mean', 'a2_z_fft_mean']
df_fft_mean.head()

,a1_x_fft_mean,a1_y_fft_mean,a1_z_fft_mean,a2_x_fft_mean,a2_y_fft_mean,a2_z_fft_mean
bearing_id,,,,,,
1,3.885181,4.124011,4.090169,2.611396,4.090169,4.124011
2,3.459109,3.662380,3.581353,2.521898,3.581353,3.662380
3,3.763797,3.986035,3.917454,2.556373,3.917454,3.986035
4,3.636525,3.834951,3.689783,2.293293,3.689783,3.834951
5,3.409107,3.702928,3.450111,2.671111,3.450111,3.702928


In [16]:
df_fft_max = df_fft.groupby('bearing_id').max()
df_fft_max.columns = ['a1_x_fft_max', 'a1_y_fft_max', 'a1_z_fft_max', 'a2_x_fft_max', 'a2_y_fft_max', 'a2_z_fft_max']
df_fft_max.head()

,a1_x_fft_max,a1_y_fft_max,a1_z_fft_max,a2_x_fft_max,a2_y_fft_max,a2_z_fft_max
bearing_id,,,,,,
1,21.134667,17.036431,18.972754,20.010540,18.972754,17.036431
2,22.177219,17.707234,15.252773,21.194466,15.252773,17.707234
3,22.810187,18.881881,16.024957,22.860777,16.024957,18.881881
4,20.557838,15.777804,16.857276,19.053852,16.857276,15.777804
5,25.201937,18.334126,16.483506,22.290534,16.483506,18.334126


In [17]:
df_fft_min = df_fft.groupby('bearing_id').min()
df_fft_min.columns = ['a1_x_fft_min', 'a1_y_fft_min', 'a1_z_fft_min', 'a2_x_fft_min', 'a2_y_fft_min', 'a2_z_fft_min']

In [18]:
df_fft_min

,a1_x_fft_min,a1_y_fft_min,a1_z_fft_min,a2_x_fft_min,a2_y_fft_min,a2_z_fft_min
bearing_id,,,,,,
1,8.220075e-05,0.017670,0.023685,0.000016,0.023685,0.017670
2,2.019833e-05,0.013191,0.022470,0.000020,0.022470,0.013191
3,2.658202e-05,0.020196,0.011786,0.000023,0.011786,0.020196
4,4.738364e-05,0.020126,0.002662,0.000009,0.002662,0.020126
5,5.730029e-05,0.020964,0.013869,0.000114,0.013869,0.020964
...,...,...,...,...,...,...
108,1.085886e-04,0.018947,0.020438,0.000021,0.020438,0.018947
109,8.197165e-06,0.010036,0.020057,0.000039,0.020057,0.010036
110,1.395479e-04,0.012807,0.018709,0.000032,0.018709,0.012807


In [19]:
df_fft_min_range = df_fft.groupby('bearing_id').min()

In [20]:
df_fft_min_range

,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
bearing_id,,,,,,
1,8.220075e-05,0.017670,0.023685,0.000016,0.023685,0.017670
2,2.019833e-05,0.013191,0.022470,0.000020,0.022470,0.013191
3,2.658202e-05,0.020196,0.011786,0.000023,0.011786,0.020196
4,4.738364e-05,0.020126,0.002662,0.000009,0.002662,0.020126
5,5.730029e-05,0.020964,0.013869,0.000114,0.013869,0.020964
...,...,...,...,...,...,...
108,1.085886e-04,0.018947,0.020438,0.000021,0.020438,0.018947
109,8.197165e-06,0.010036,0.020057,0.000039,0.020057,0.010036
110,1.395479e-04,0.012807,0.018709,0.000032,0.018709,0.012807


In [21]:
df_fft_max_range = df_fft.groupby('bearing_id').max()
df_fft_max_range

,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
bearing_id,,,,,,
1,21.134667,17.036431,18.972754,20.010540,18.972754,17.036431
2,22.177219,17.707234,15.252773,21.194466,15.252773,17.707234
3,22.810187,18.881881,16.024957,22.860777,16.024957,18.881881
4,20.557838,15.777804,16.857276,19.053852,16.857276,15.777804
5,25.201937,18.334126,16.483506,22.290534,16.483506,18.334126
...,...,...,...,...,...,...
108,23.001944,18.745835,20.224667,21.399427,20.224667,18.745835
109,20.626941,16.873323,19.689308,17.409621,19.689308,16.873323
110,23.248090,14.512553,21.281811,22.410722,21.281811,14.512553


In [22]:
df_fft_range = df_fft_max_range.subtract(df_fft_min_range, fill_value=0)
df_fft_range

,a1_x_fft,a1_y_fft,a1_z_fft,a2_x_fft,a2_y_fft,a2_z_fft
bearing_id,,,,,,
1,21.134584,17.018761,18.949070,20.010524,18.949070,17.018761
2,22.177199,17.694042,15.230303,21.194446,15.230303,17.694042
3,22.810161,18.861685,16.013171,22.860754,16.013171,18.861685
4,20.557791,15.757678,16.854614,19.053842,16.854614,15.757678
5,25.201880,18.313163,16.469636,22.290420,16.469636,18.313163
...,...,...,...,...,...,...
108,23.001836,18.726888,20.204229,21.399406,20.204229,18.726888
109,20.626933,16.863286,19.669251,17.409582,19.669251,16.863286
110,23.247951,14.499746,21.263101,22.410690,21.263101,14.499746


In [23]:
df_fft_range.columns = ['a1_x_ff_range', 'a1_y_fft_range', 'a1_z_fft_range', 'a2_x_fft_range', 'a2_y_fft_range', 'a2_z_fft_range']

In [24]:
df_fft_master = (pd.merge(df_fft_mean, df_fft_range, on='bearing_id'))
df_fft_master = (pd.merge(df_fft_master, df_fft_min, on='bearing_id'))
df_fft_master = (pd.merge(df_fft_master, df_fft_max, on='bearing_id'))
df_fft_master

,a1_x_fft_mean,a1_y_fft_mean,a1_z_fft_mean,a2_x_fft_mean,a2_y_fft_mean,a2_z_fft_mean,a1_x_ff_range,a1_y_fft_range,a1_z_fft_range,a2_x_fft_range,...,a1_z_fft_min,a2_x_fft_min,a2_y_fft_min,a2_z_fft_min,a1_x_fft_max,a1_y_fft_max,a1_z_fft_max,a2_x_fft_max,a2_y_fft_max,a2_z_fft_max
bearing_id,,,,,,,,,,,,,,,,,,,,,
1,3.885181,4.124011,4.090169,2.611396,4.090169,4.124011,21.134584,17.018761,18.949070,20.010524,...,0.023685,0.000016,0.023685,0.017670,21.134667,17.036431,18.972754,20.010540,18.972754,17.036431
2,3.459109,3.662380,3.581353,2.521898,3.581353,3.662380,22.177199,17.694042,15.230303,21.194446,...,0.022470,0.000020,0.022470,0.013191,22.177219,17.707234,15.252773,21.194466,15.252773,17.707234
3,3.763797,3.986035,3.917454,2.556373,3.917454,3.986035,22.810161,18.861685,16.013171,22.860754,...,0.011786,0.000023,0.011786,0.020196,22.810187,18.881881,16.024957,22.860777,16.024957,18.881881
4,3.636525,3.834951,3.689783,2.293293,3.689783,3.834951,20.557791,15.757678,16.854614,19.053842,...,0.002662,0.000009,0.002662,0.020126,20.557838,15.777804,16.857276,19.053852,16.857276,15.777804
5,3.409107,3.702928,3.450111,2.671111,3.450111,3.702928,25.201880,18.313163,16.469636,22.290420,...,0.013869,0.000114,0.013869,0.020964,25.201937,18.334126,16.483506,22.290534,16.483506,18.334126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,3.064386,3.589980,3.637390,3.014652,3.637390,3.589980,23.001836,18.726888,20.204229,21.399406,...,0.020438,0.000021,0.020438,0.018947,23.001944,18.745835,20.224667,21.399427,20.224667,18.745835
109,2.534319,2.619303,3.406991,2.127519,3.406991,2.619303,20.626933,16.863286,19.669251,17.409582,...,0.020057,0.000039,0.020057,0.010036,20.626941,16.873323,19.689308,17.409621,19.689308,16.873323
110,3.289674,2.912930,3.709078,2.505268,3.709078,2.912930,23.247951,14.499746,21.263101,22.410690,...,0.018709,0.000032,0.018709,0.012807,23.248090,14.512553,21.281811,22.410722,21.281811,14.512553


In [25]:
df_final_master = (pd.merge(df_master, df_fft_master, on='bearing_id'))


In [26]:
df_final_master

,a1_x_mean,a1_y_mean,a1_z_mean,a2_x_mean,a2_y_mean,a2_z_mean,rpm_mean,hz_mean,w_mean,status,...,a1_z_fft_min,a2_x_fft_min,a2_y_fft_min,a2_z_fft_min,a1_x_fft_max,a1_y_fft_max,a1_z_fft_max,a2_x_fft_max,a2_y_fft_max,a2_z_fft_max
bearing_id,,,,,,,,,,,,,,,,,,,,,
1,-0.176971,-0.001740,-0.035361,0.114590,0.023329,0.250710,1082.423656,18.040394,0.419961,0,...,0.023685,0.000016,0.023685,0.017670,21.134667,17.036431,18.972754,20.010540,18.972754,17.036431
2,-0.174032,-0.007636,-0.053353,-0.178986,0.024936,0.194061,1074.981634,17.916361,0.501160,0,...,0.022470,0.000020,0.022470,0.013191,22.177219,17.707234,15.252773,21.194466,15.252773,17.707234
3,-0.172328,-0.002343,-0.039898,-0.121842,0.036149,0.183881,1149.927038,19.165451,0.513051,0,...,0.011786,0.000023,0.011786,0.020196,22.810187,18.881881,16.024957,22.860777,16.024957,18.881881
4,-0.149644,-0.000549,-0.037679,-0.119574,0.030838,0.200699,1127.538630,18.792311,0.539426,0,...,0.002662,0.000009,0.002662,0.020126,20.557838,15.777804,16.857276,19.053852,16.857276,15.777804
5,-0.146438,0.015268,-0.026678,-0.080799,0.034042,0.206547,1037.216452,17.286941,0.353715,0,...,0.013869,0.000114,0.013869,0.020964,25.201937,18.334126,16.483506,22.290534,16.483506,18.334126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,-0.145112,0.026677,0.120057,-0.225940,0.069918,0.219440,952.405287,15.873421,0.387498,1,...,0.020438,0.000021,0.020438,0.018947,23.001944,18.745835,20.224667,21.399427,20.224667,18.745835
109,-0.124200,0.077907,0.219025,-0.229740,0.099719,0.184505,1000.562215,16.676037,0.262752,1,...,0.020057,0.000039,0.020057,0.010036,20.626941,16.873323,19.689308,17.409621,19.689308,16.873323
110,-0.132581,0.080428,0.213349,-0.209128,0.117938,0.079041,1073.939895,17.898998,0.302760,1,...,0.018709,0.000032,0.018709,0.012807,23.248090,14.512553,21.281811,22.410722,21.281811,14.512553


In [27]:
df_final_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 1 to 112
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   a1_x_mean       111 non-null    float64
 1   a1_y_mean       111 non-null    float64
 2   a1_z_mean       111 non-null    float64
 3   a2_x_mean       111 non-null    float64
 4   a2_y_mean       111 non-null    float64
 5   a2_z_mean       111 non-null    float64
 6   rpm_mean        111 non-null    float64
 7   hz_mean         111 non-null    float64
 8   w_mean          111 non-null    float64
 9   status          111 non-null    int64  
 10  a1_x_range      111 non-null    float64
 11  a1_y_range      111 non-null    float64
 12  a1_z_range      111 non-null    float64
 13  a2_x_range      111 non-null    float64
 14  a2_y_range      111 non-null    float64
 15  a2_z_range      111 non-null    float64
 16  w_range         111 non-null    float64
 17  a1_x_min        111 non-null    flo